<a href="https://colab.research.google.com/github/chaeyh4/Freezing/blob/main/Freezing_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd
import os

import matplotlib.pyplot as plt

# Data

## CSV Creation

In [4]:
def get_txt(path): #txt files in path
  txt_files = []
  for file in os.listdir(path):
    if file.endswith(".txt"):
      txt_files.append(file)
  return txt_files

In [5]:
path = "/content/drive/MyDrive/daphnet+freezing+of+gait/dataset_fog_release/dataset"
txt_files = get_txt(path)
print(txt_files)

['S01R01.txt', 'S01R02.txt', 'S02R01.txt', 'S02R02.txt', 'S03R01.txt', 'S03R02.txt', 'S03R03.txt', 'S04R01.txt', 'S05R01.txt', 'S05R02.txt', 'S06R01.txt', 'S06R02.txt', 'S07R01.txt', 'S07R02.txt', 'S08R01.txt', 'S09R01.txt', 'S10R01.txt']


In [27]:
txt_data = []

for file in txt_files:
    file_path = os.path.join(path, file)
    df = pd.read_csv(file_path, delimiter=' ', header=None, names=['Time(ms)', 'ankle_acc_horiz_forward', 'ankle_acc_vertical', 'ankle_acc_horiz_lateral', 'upper_leg_acc_horiz_forward', 'upper_leg_acc_vertical', 'upper_leg_acc_horiz_lateral', 'trunk_acc_horiz_forward', 'trunk_acc_vertical', 'trunk_acc_horiz_lateral', 'annotations'])

    user_num = int(file.split("R")[0][1:])
    run_num = int(file.split("R")[1].split(".")[0])

    df['user'] = user_num
    df['run'] = run_num

    txt_data.append(df)

total_df = pd.concat(txt_data, ignore_index=True)
total_df = total_df[['user', 'run', 'Time(ms)', 'ankle_acc_horiz_forward', 'ankle_acc_vertical',
       'ankle_acc_horiz_lateral', 'upper_leg_acc_horiz_forward',
       'upper_leg_acc_vertical', 'upper_leg_acc_horiz_lateral',
       'trunk_acc_horiz_forward', 'trunk_acc_vertical',
       'trunk_acc_horiz_lateral', 'annotations']]

In [28]:
total_df.to_csv("/content/drive/MyDrive/daphnet+freezing+of+gait/total_data.csv", index=False)

## Read CSV

In [3]:
total_df = pd.read_csv("/content/drive/MyDrive/daphnet+freezing+of+gait/total_data.csv")

In [4]:
total_df.columns

Index(['user', 'run', 'Time(ms)', 'ankle_acc_horiz_forward',
       'ankle_acc_vertical', 'ankle_acc_horiz_lateral',
       'upper_leg_acc_horiz_forward', 'upper_leg_acc_vertical',
       'upper_leg_acc_horiz_lateral', 'trunk_acc_horiz_forward',
       'trunk_acc_vertical', 'trunk_acc_horiz_lateral', 'annotations'],
      dtype='object')

In [5]:
total_df

,user,run,Time(ms),ankle_acc_horiz_forward,ankle_acc_vertical,ankle_acc_horiz_lateral,upper_leg_acc_horiz_forward,upper_leg_acc_vertical,upper_leg_acc_horiz_lateral,trunk_acc_horiz_forward,trunk_acc_vertical,trunk_acc_horiz_lateral,annotations
0,1,1,15,70,39,-970,0,0,0,0,0,0,0
1,1,1,31,70,39,-970,0,0,0,0,0,0,0
2,1,1,46,60,49,-960,0,0,0,0,0,0,0
3,1,1,62,60,49,-960,0,0,0,0,0,0,0
4,1,1,78,50,39,-960,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1917882,10,1,3020296,-131,107,-960,0,0,0,0,0,0,0
1917883,10,1,3020312,-121,127,-970,0,0,0,0,0,0,0
1917884,10,1,3020328,-141,117,-960,0,0,0,0,0,0,0
1917885,10,1,3020343,-131,127,-980,0,0,0,0,0,0,0


## Filtering -> Normalizng



In [6]:
from scipy.signal import butter, filtfilt

# butter_bandpass
def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a

# bandpass_filter
def bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = filtfilt(b, a, data)
    return y

# Normalize function
def normalize_signal(signal):
    return signal / signal.max() if signal.max() != 0 else signal

# Preprocess
def preprocess_signals(df):
    fs = 64
    lowcut = 0.1
    highcut = 15.0

    # filtering
    df['ankle_x'] = bandpass_filter(df['ankle_acc_horiz_forward'], lowcut, highcut, fs)
    df['ankle_y'] = bandpass_filter(df['ankle_acc_vertical'], lowcut, highcut, fs)
    df['ankle_z'] = bandpass_filter(df['ankle_acc_horiz_lateral'], lowcut, highcut, fs)

    df['upper_leg_x'] = bandpass_filter(df['upper_leg_acc_horiz_forward'], lowcut, highcut, fs)
    df['upper_leg_y'] = bandpass_filter(df['upper_leg_acc_vertical'], lowcut, highcut, fs)
    df['upper_leg_z'] = bandpass_filter(df['upper_leg_acc_horiz_lateral'], lowcut, highcut, fs)

    df['trunk_x'] = bandpass_filter(df['trunk_acc_horiz_forward'], lowcut, highcut, fs)
    df['trunk_y'] = bandpass_filter(df['trunk_acc_vertical'], lowcut, highcut, fs)
    df['trunk_z'] = bandpass_filter(df['trunk_acc_horiz_lateral'], lowcut, highcut, fs)

    # magnitude
    df['ankle_magnitude'] = (df['ankle_x']**2 + df['ankle_y']**2 + df['ankle_z']**2)**0.5
    df['upper_leg_magnitude'] = (df['upper_leg_x']**2 + df['upper_leg_y']**2 + df['upper_leg_z']**2)**0.5
    df['trunk_magnitude'] = (df['trunk_x']**2 + df['trunk_y']**2 + df['trunk_z']**2)**0.5

    # normalize signals
    df['ankle_x_normalized'] = normalize_signal(df['ankle_x'])
    df['ankle_y_normalized'] = normalize_signal(df['ankle_y'])
    df['ankle_z_normalized'] = normalize_signal(df['ankle_z'])

    df['upper_leg_x_normalized'] = normalize_signal(df['upper_leg_x'])
    df['upper_leg_y_normalized'] = normalize_signal(df['upper_leg_y'])
    df['upper_leg_z_normalized'] = normalize_signal(df['upper_leg_z'])

    df['trunk_x_normalized'] = normalize_signal(df['trunk_x'])
    df['trunk_y_normalized'] = normalize_signal(df['trunk_y'])
    df['trunk_z_normalized'] = normalize_signal(df['trunk_z'])

    # normalize magnitudes
    df['ankle_magnitude_normalized'] = normalize_signal(df['ankle_magnitude'])
    df['upper_leg_magnitude_normalized'] = normalize_signal(df['upper_leg_magnitude'])
    df['trunk_magnitude_normalized'] = normalize_signal(df['trunk_magnitude'])

    return df

total_df = preprocess_signals(total_df)


In [7]:
total_df

,user,run,Time(ms),ankle_acc_horiz_forward,ankle_acc_vertical,ankle_acc_horiz_lateral,upper_leg_acc_horiz_forward,upper_leg_acc_vertical,upper_leg_acc_horiz_lateral,trunk_acc_horiz_forward,...,ankle_z_normalized,upper_leg_x_normalized,upper_leg_y_normalized,upper_leg_z_normalized,trunk_x_normalized,trunk_y_normalized,trunk_z_normalized,ankle_magnitude_normalized,upper_leg_magnitude_normalized,trunk_magnitude_normalized
0,1,1,15,70,39,-970,0,0,0,0,...,0.000043,1.011047e-05,-1.742989e-05,7.854260e-04,8.457877e-04,7.615276e-05,-7.534968e-03,0.000234,4.024292e-04,1.597957e-03
1,1,1,31,70,39,-970,0,0,0,0,...,0.000106,1.151752e-05,-1.983641e-05,8.718161e-04,8.591072e-04,7.737091e-05,-7.654547e-03,0.000324,4.467047e-04,1.623314e-03
2,1,1,46,60,49,-960,0,0,0,0,...,0.000294,1.293765e-05,-2.226524e-05,9.589867e-04,8.724248e-04,7.858900e-05,-7.774116e-03,0.000459,4.913805e-04,1.648668e-03
3,1,1,62,60,49,-960,0,0,0,0,...,0.000509,1.437084e-05,-2.471639e-05,1.046938e-03,8.857391e-04,7.980690e-05,-7.893660e-03,0.000620,5.364565e-04,1.674017e-03
4,1,1,78,50,39,-960,0,0,0,0,...,0.000487,1.581711e-05,-2.718985e-05,1.135669e-03,8.990488e-04,8.102449e-05,-8.013169e-03,0.000696,5.819326e-04,1.699359e-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1917882,10,1,3020296,-131,107,-960,0,0,0,0,...,-0.026256,-2.310324e-45,9.829745e-46,-3.018120e-44,2.820809e-44,-8.102098e-45,1.284617e-43,0.015356,1.556260e-44,2.889856e-44
1917883,10,1,3020312,-121,127,-970,0,0,0,0,...,-0.031348,-2.277207e-45,9.688886e-46,-2.974862e-44,2.794158e-44,-8.024593e-45,1.272379e-43,0.018340,1.533955e-44,2.862329e-44
1917884,10,1,3020328,-141,117,-960,0,0,0,0,...,-0.034663,-2.242374e-45,9.540719e-46,-2.929361e-44,2.764847e-44,-7.939484e-45,1.258932e-43,0.020288,1.510493e-44,2.832084e-44
1917885,10,1,3020343,-131,127,-980,0,0,0,0,...,-0.023519,-2.205805e-45,9.385170e-46,-2.881593e-44,2.732832e-44,-7.846653e-45,1.244260e-43,0.013791,1.485862e-44,2.799081e-44


## Preprocessing

In [8]:
total_df['annotations'].value_counts()

,count
annotations,
1,1030050
0,777052
2,110785


In [9]:
total_df.columns

Index(['user', 'run', 'Time(ms)', 'ankle_acc_horiz_forward',
       'ankle_acc_vertical', 'ankle_acc_horiz_lateral',
       'upper_leg_acc_horiz_forward', 'upper_leg_acc_vertical',
       'upper_leg_acc_horiz_lateral', 'trunk_acc_horiz_forward',
       'trunk_acc_vertical', 'trunk_acc_horiz_lateral', 'annotations',
       'ankle_x', 'ankle_y', 'ankle_z', 'upper_leg_x', 'upper_leg_y',
       'upper_leg_z', 'trunk_x', 'trunk_y', 'trunk_z', 'ankle_magnitude',
       'upper_leg_magnitude', 'trunk_magnitude', 'ankle_x_normalized',
       'ankle_y_normalized', 'ankle_z_normalized', 'upper_leg_x_normalized',
       'upper_leg_y_normalized', 'upper_leg_z_normalized',
       'trunk_x_normalized', 'trunk_y_normalized', 'trunk_z_normalized',
       'ankle_magnitude_normalized', 'upper_leg_magnitude_normalized',
       'trunk_magnitude_normalized'],
      dtype='object')

In [21]:
train_df = total_df[total_df['user'].isin([1, 2, 3, 4, 5, 6, 7, 8])]
val_df = total_df[total_df['user'].isin([8])]
test_df = total_df[total_df['user'].isin([9])]

In [22]:
import numpy as np
import pandas as pd

def segmentation(df, segment_size=320):
    segments = []
    grouped = df.groupby(['user', 'run'])

    for (user, run), group in grouped:
        for start in range(0, len(group), segment_size):
            segment = group.iloc[start:start + segment_size]
            if len(segment) == segment_size:
                segments.append(segment)

    segments_array = []
    labels_array = []

    for segment in segments:
        X_segment = segment[['ankle_x_normalized', 'ankle_y_normalized', 'ankle_z_normalized',
                             'upper_leg_x_normalized', 'upper_leg_y_normalized', 'upper_leg_z_normalized',
                             'trunk_x_normalized', 'trunk_y_normalized', 'trunk_z_normalized',
                             'ankle_magnitude_normalized', 'upper_leg_magnitude_normalized',
                             'trunk_magnitude_normalized']].values

        y_segment = segment[['annotations']].values.flatten()

        unique, counts = np.unique(y_segment, return_counts=True)
        frequencies = dict(zip(unique, counts))

        if 0 in frequencies and frequencies[0] > (segment_size / 2):
            continue

        label = None
        for value, count in frequencies.items():
            if count > (segment_size / 2):
                label = value
                break

        if label is not None:
            if label == 2:
                label = 1
            elif label == 1:
                label = 0

            segments_array.append(X_segment)
            labels_array.append(label)

    segments_array = np.array(segments_array)
    labels_array = np.array(labels_array)

    return segments_array, labels_array

X_train, y_train = segmentation(train_df)
X_val, y_val = segmentation(val_df)
X_test, y_test = segmentation(test_df)

print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"X_val shape: {X_val.shape}")
print(f"y_val shape: {y_val.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_test shape: {y_test.shape}")


X_train shape: (2769, 320, 12)
y_train shape: (2769,)
X_val shape: (153, 320, 12)
y_val shape: (153,)
X_test shape: (348, 320, 12)
y_test shape: (348,)


In [23]:
def label_frequencies(labels, dataset_name):
    unique, counts = np.unique(labels, return_counts=True)
    frequencies = dict(zip(unique, counts))

    print(f"{dataset_name} label frequencies:")
    for label, count in frequencies.items():
        print(f"Label: {label}, Count: {count}")

label_frequencies(y_train, "Training")
label_frequencies(y_val, "Validation")
label_frequencies(y_test, "Test")

Training label frequencies:
Label: 0, Count: 2494
Label: 1, Count: 275
Validation label frequencies:
Label: 0, Count: 112
Label: 1, Count: 41
Test label frequencies:
Label: 0, Count: 298
Label: 1, Count: 50


In [24]:
# Balancing

label_0_indices = np.where(y_train == 0)[0]
label_1_indices = np.where(y_train == 1)[0]

num_label_1 = len(label_1_indices)
label_0_sample_indices = np.random.choice(label_0_indices, size=num_label_1, replace=False)

final_indices = np.concatenate([label_0_sample_indices, label_1_indices])

X_train_balanced = X_train[final_indices]
y_train_balanced = y_train[final_indices]

print(f'Balanced Training Set Size: {len(y_train_balanced)}')
print(f'Label 0 Count: {np.sum(y_train_balanced == 0)}')
print(f'Label 1 Count: {np.sum(y_train_balanced == 1)}')


Balanced Training Set Size: 550
Label 0 Count: 275
Label 1 Count: 275


# Model

In [25]:
import tensorflow as tf
from tensorflow.keras import layers, models

def fog_detection_model(input_shape):
    model = models.Sequential()

    # Conv1D, Batch Normalization, MaxPooling
    model.add(layers.Conv1D(256, kernel_size=5, activation='relu', input_shape=input_shape))
    model.add(layers.MaxPooling1D(pool_size=2))

    # Conv1D, Batch Normalization
    model.add(layers.Conv1D(128, kernel_size=5, activation='relu'))
    model.add(layers.BatchNormalization())

    # 3x Conv1D, Batch Normalization
    for _ in range(3):
        model.add(layers.Conv1D(64, kernel_size=5, activation='relu'))
        model.add(layers.BatchNormalization())

    # MaxPooling
    model.add(layers.MaxPooling1D(pool_size=2))

    # Flatten
    model.add(layers.Flatten())

    # Dropout
    model.add(layers.Dropout(0.5))

    # Dense Layers
    model.add(layers.Dense(140, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(70, activation='relu'))
    model.add(layers.Dense(2, activation='softmax'))  # Two classes for binary classification

    return model

input_shape = (320, 12)  # (time_steps, features)
model = fog_detection_model(input_shape)

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',  # Change to sparse categorical crossentropy
              metrics=['accuracy'])

model.summary()

# 모델 훈련
history = model.fit(X_train_balanced, y_train_balanced,
                    validation_data=(X_val, y_val),
                    batch_size=32,
                    epochs=50)

# 모델 평가
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv1d_10 (Conv1D)                   │ (None, 316, 256)            │          15,616 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_4 (MaxPooling1D)       │ (None, 158, 256)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_11 (Conv1D)                   │ (None, 154, 128)            │         163,968 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_8                │ (None, 154, 128)            │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_12 (Conv1D)                   │ (None, 150, 64)             │          41,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_9                │ (None, 150, 64)             │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_13 (Conv1D)                   │ (None, 146, 64)             │          20,544 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_10               │ (None, 146, 64)             │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_14 (Conv1D)                   │ (None, 142, 64)             │          20,544 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_11               │ (None, 142, 64)             │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_5 (MaxPooling1D)       │ (None, 71, 64)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_2 (Flatten)                  │ (None, 4544)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ (None, 4544)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 140)                 │         636,300 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_5 (Dropout)                  │ (None, 140)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 70)                  │           9,870 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (None, 2)                   │             142 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 909,288 (3.47 MB)

 Trainable params: 908,648 (3.47 MB)

 Non-trainable params: 640 (2.50 KB)

Epoch 1/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 10s 322ms/step - accuracy: 0.5996 - loss: 1.2604 - val_accuracy: 0.7320 - val_loss: 0.6233
Epoch 2/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 8s 207ms/step - accuracy: 0.7431 - loss: 0.6086 - val_accuracy: 0.7320 - val_loss: 0.5850
Epoch 3/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 5s 287ms/step - accuracy: 0.8374 - loss: 0.3882 - val_accuracy: 0.7320 - val_loss: 0.6553
Epoch 4/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 9s 206ms/step - accuracy: 0.8587 - loss: 0.3678 - val_accuracy: 0.7320 - val_loss: 0.7111
Epoch 5/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 7s 326ms/step - accuracy: 0.9073 - loss: 0.2729 - val_accuracy: 0.7320 - val_loss: 0.8078
Epoch 6/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 8s 211ms/step - accuracy: 0.9031 - loss: 0.2406 - val_accuracy: 0.7320 - val_loss: 0.7564
Epoch 7/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 7s 333ms/step - accuracy: 0.9290 - loss: 0.2322 - val_accuracy: 0.7320 - val_loss: 0.9839
Epoch 8/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 4s 204ms/step - accuracy: 0.9210 - loss: 0.2105 - val_accuracy: 0

In [30]:
from sklearn.metrics import classification_report, confusion_matrix
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)

print(classification_report(y_test, y_pred_classes))

confusion = confusion_matrix(y_test, y_pred_classes)
print("Confusion Matrix:")
print(confusion)

11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 90ms/step
              precision    recall  f1-score   support

           0       0.97      0.96      0.96       298
           1       0.75      0.80      0.78        50

    accuracy                           0.93       348
   macro avg       0.86      0.88      0.87       348
weighted avg       0.94      0.93      0.93       348

Confusion Matrix:
[[285  13]
 [ 10  40]]
